In [1]:
# [1]
!pip install datasets
!pip install scikit-learn
!pip install ibm-watson-machine-learning==1.0.312

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 75.6 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm_watson_machine_learning 1.0.367
    Uninstalling ibm_watson_machine_learning-1.0.367:
      Successfully uninstalled ibm_watson_machine_learning-1.0.367
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autoai-libs 2.0.19 requires pandas==2.1.*, but you have pandas 1.5.3 which is incompatible.
autoai-ts-libs 4.0.16 requires pandas==2.1.*, but you have pandas 1.5.3 which is incompatible.


In [2]:
# [2]
import os, getpass
from pandas import read_csv

In [3]:
# [3] Watsonx API connection
# Enter the URL for Watson Machine Learning as per your WML Location
# Dallas : us-south

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

Please enter your WML api key (hit enter):  ········


In [6]:
# [4] Defining the project id
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

In [5]:
# [8]
project_id

'ca947bfe-a770-4ce7-b4a0-de1f521e439d'

In [ ]:
# [9]
import os, types
import pandas as pd
from ibm_boto3 import client
from botocore.client import Config
import ibm_boto3

def _iter_(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. 
# It includes your credentials. 
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Enter your API KEY HERE',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'bucketexpt2-testandtrain'
object_key = 'hotel_train.csv'

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Add missing _iter_ method so pandas accepts body as file-like object
if not hasattr(body, "_iter"): body.iter_ = types.MethodType(_iter_, body)

train_data = pd.read_csv(body)
train_data.head(5)


,Review,Sentiment,Tags
0,TV wasn't working and maintenance was delayed.,Neutral,Location
1,Worst stay ever. Broken lights and no hot water.,Positive,Room Service
2,Rooms were cozy and had good lighting.,Negative,"Wi-Fi, Noise Levels, Restaurant"
3,Outstanding hospitality and great service.,Negative,Service
4,"Room service was slow, but the location is per...",Positive,"Amenities, Food Quality, Wi-Fi"


In [ ]:
# [10]
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def _iter_(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. 
# It includes your credentials. 
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ENTER YOUT API KEY HERE',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url="https://s3.private.us-south.cloud-object-storage.appdomain.cloud"
)

bucket = 'bucketexpt2-testandtrain'
object_key = 'hotel_test.csv'

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# add missing _iter_ method, so pandas accepts body as file-like object
if not hasattr(body, "_iter"): body.iter_ = types.MethodType(_iter_, body)

test_data = pd.read_csv(body)
test_data.head(5)


,Review,Sentiment,Tags
0,The view from the room was breathtaking. Loved...,Neutral,"Spa, Pool, Noise Levels"
1,The restaurant served cold food.,Neutral,"Staff Behavior, Wi-Fi, Noise Levels"
2,Excellent service and very friendly staff. Lov...,Neutral,"Wi-Fi, Pool, Location"
3,"Room service was slow, but the location is per...",Negative,"Check-in/Check-out, Bathroom"
4,The breakfast had many healthy options.,Positive,Pool


In [9]:
# [29]
train_data.shape

(50, 3)

In [10]:
# [30]
test_data.shape

(50, 3)

In [11]:
# [13]
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

In [12]:
# [14]
model_id = ModelTypes.FLAN_UL2

In [ ]:
satisfaction_instruction = """
Was the hotel review positive or negative? Respond with 1 for positive, 0 for negative.

review: The hotel was dirty and noisy.
sentiment: 0

review: The staff was friendly and the room was spotless.
sentiment: 1

review:
"""

In [20]:
import time

In [21]:
# [17]
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)


In [25]:
results = []
comments = list(test_data.Review)

for idx, input_text in enumerate(comments):
    prompt = satisfaction_instruction + input_text + "\nsentiment:"
    try:
        output = model.generate_text(prompt=prompt)
        results.append(output.strip())
    except Exception as e:
        print(f"⚠️ Error at row {idx}: {e}")
        results.append("ERROR")
    time.sleep(0.6)  # Prevent rate-limit (2 req/sec for Lite plan)


In [26]:
test_data["Predicted_Sentiment"] = results


In [27]:
print("\n✅ Final Results:")
print(test_data[["Review", "Sentiment", "Predicted_Sentiment"]].head())


✅ Final Results:
                                              Review Sentiment  \
0  The view from the room was breathtaking. Loved...   Neutral   
1                   The restaurant served cold food.   Neutral   
2  Excellent service and very friendly staff. Lov...   Neutral   
3  Room service was slow, but the location is per...  Negative   
4            The breakfast had many healthy options.  Positive   

  Predicted_Sentiment  
0                   1  
1                   0  
2                   1  
3                   1  
4                   1  


In [28]:
test_data.to_csv("hotel_sentiment_predictions.csv", index=False)
print("\n📁 Results saved to hotel_sentiment_predictions.csv")


📁 Results saved to hotel_sentiment_predictions.csv


In [29]:
test_data["Predicted_Sentiment"] = results
print("\n✅ Final Predictions:\n")
print(test_data[["Review", "Sentiment", "Predicted_Sentiment"]].head(15))


✅ Final Predictions:

                                               Review Sentiment  \
0   The view from the room was breathtaking. Loved...   Neutral   
1                    The restaurant served cold food.   Neutral   
2   Excellent service and very friendly staff. Lov...   Neutral   
3   Room service was slow, but the location is per...  Negative   
4             The breakfast had many healthy options.  Positive   
5   Not satisfied with the housekeeping. Towels we...   Neutral   
6              Friendly staff and quick room service.  Positive   
7        The breakfast buffet was not up to the mark.   Neutral   
8   Staff was unhelpful and rude. Not coming back ...  Positive   
9          Lovely aroma in the room and clean linens.   Neutral   
10           Staff forgot to clean our room on day 2.  Negative   
11  The room was clean and spacious, but the food ...   Neutral   
12  Wi-Fi was very slow, but everything else was f...  Negative   
13                  Enjoyed the complim